In [35]:
EVERYONE_TEXT = 'everyone'

import re
import typing
from collections import defaultdict

In [36]:
class Item:
    def __init__(self, name: str, cost: float, people_to_pay: typing.List=[]) -> None:
        self.name = name.strip().lower()
        self.cost = cost
        self.people_to_pay = people_to_pay

    def assign_to_person(self) -> None:
        cost_per_person = self.cost / len(self.people_to_pay)

        for person in self.people_to_pay:
            person.total_due += cost_per_person
            person.items.append(self)

    def __repr__(self) -> str:
        return self.__str__()

    def __str__(self) -> str:
        return f'Item {self.name.capitalize()} with cost {self.cost} and buyers {[person.name.capitalize() for person in self.people_to_pay]}'

class Person:
    def __init__(self, name: str) -> None:
        self.name = name.strip().lower()
        self.items = []
        self.total_due = 0.0

    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        items_with_fractions_formatted = [f'1/{len(item.people_to_pay)} of {item.name.capitalize()}' for item in self.items]
        return f'{self.name.capitalize()}, in charge of {items_with_fractions_formatted} with a total cost of {round(self.total_due, 2)}'

In [41]:
def remove_currency_symbols(cost_with_currency: str) -> str:
    return re.sub(r'[^\d.,]+', '', cost_with_currency)

def get_people_in_line(formatted_people: str, people: typing.List[Person]) -> typing.List[Person]:
    people_in_line = [person.lower() for person in re.findall(r'(\w+)', formatted_people)]

    if 'everyone' in people_in_line:
        return people

    extracted_people = [person for person in people if person.name in people_in_line]

    if len(people_in_line) != len(extracted_people):
        people_not_found = [person.name for person in people if person.name not in people_in_line]
        raise BaseException(f'Some people could not be found in the list of names that you provided: {people_not_found}')

    return extracted_people


def get_line_items(line: str, people: typing.List[Person]) -> typing.List[str]:
    pattern = r'(.+): (.+) \((.+)\)'

    matches = re.findall(pattern, line)
    
    if len(matches) == 0:
        raise BaseException(f'Incorrectly formatted line: {pattern}')

    line_items = matches[0]

    if len(line_items) != 3:
        raise BaseException(f'Incorrectly formatted line: {pattern}')
    
    return [
        line_items[0],
        float(remove_currency_symbols(line_items[1])),
        get_people_in_line(line_items[2], people)
    ]

def get_item_from_line(line: str, people: typing.List[Person]) -> Item:
    item, cost, people_to_pay = get_line_items(line, people)
    return Item(name=item, cost=cost, people_to_pay=people_to_pay)

def get_all_items_from_file(fp: str, people: typing.List[Person]) -> typing.List[Item]:
    with open(fp, 'r') as file:
        lines = file.readlines()
        return [get_item_from_line(line, people) for line in lines]

In [38]:
person_names = input('Please enter all names on this tab, separated by a comma.')

person_names = [name for name in person_names.split(',')]
people = [Person(name=name) for name in person_names]

items = get_all_items_from_file('./tabs/costco/costco-tab-08142022.txt', people)

In [39]:
for item in items:
    item.assign_to_person()

In [40]:
people

[Jared, in charge of ['1/1 of Lemon juice', '1/3 of Toilet paper', '1/3 of Paper towels', '1/3 of Rinse aid', '1/3 of Dish soap', '1/3 of Bounce dryer sheets', '1/3 of Magic erasers', '1/3 of Steaks', '1/1 of Gillette razors', '1/1 of Degree deodorant', '1/3 of Clorox wand', '1/3 of Clorox wipes', '1/3 of Downy scent beads', '1/3 of Bath mat', '1/3 of Tax'] with a total cost of 135.41,
 Carter, in charge of ['1/3 of Toilet paper', '1/3 of Paper towels', '1/1 of Peaches', '1/3 of Rinse aid', '1/3 of Dish soap', '1/3 of Bounce dryer sheets', '1/3 of Magic erasers', '1/3 of Steaks', '1/1 of Cremo shave cream', '1/1 of Degree deodorant', '1/3 of Clorox wand', '1/3 of Clorox wipes', '1/3 of Downy scent beads', '1/3 of Bath mat', '1/2 of Bath mat', '1/3 of Tax'] with a total cost of 112.91,
 Taimur, in charge of ['1/3 of Toilet paper', '1/3 of Paper towels', '1/3 of Rinse aid', '1/3 of Dish soap', '1/3 of Bounce dryer sheets', '1/3 of Magic erasers', '1/3 of Steaks', '1/3 of Clorox wand', '1